In [2]:
# Initialization
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [1]:
n = 1000

# Vector of states
states = np.repeat("S",n)
states[0] = "I"

# Probabilities of entering 
prob_S_I = 0.9
prob_I_R = 0.3
prob_R_S = 0.015

n_S = np.array([len(np.where(states == "S")[0])])
n_I = np.array([len(np.where(states == "I")[0])])
n_R = np.array([len(np.where(states == "R")[0])])


while "I" in states:
    
    # Get positions of S, I, R
    S_index = np.where(states == "S")[0]
    I_index = np.where(states == "I")[0]
    R_index = np.where(states == "R")[0]

    # Update susceptible
    prob_S_I_t = prob_S_I * len(I_index)/n
    states[S_index] = np.random.choice(np.array(["S","I"]),p = np.array([1-prob_S_I_t, prob_S_I_t]), size = len(S_index))


    # Update infected
    states[I_index] = np.random.choice(np.array(["I", "R"]),p = np.array([1-prob_I_R, prob_I_R]), size = len(I_index))


    # Update recovered
    states[R_index] = np.random.choice(np.array(["R", "S"]),p = np.array([1-prob_R_S, prob_R_S]), size = len(R_index))

    
    n_S = np.concatenate([n_S, np.array([len(np.where(states == "S")[0])])])
    n_I = np.concatenate([n_I, np.array([len(np.where(states == "I")[0])])])
    n_R = np.concatenate([n_R, np.array([len(np.where(states == "R")[0])])])

NameError: name 'np' is not defined

In [ ]:
#print(states)
#print(n_S)
#print(n_I)
#print(n_R)

plt.figure()
plt.plot(n_S, "-b", label = "Susceptible")
plt.plot(n_I, "-r", label = "Infected")
plt.plot(n_R, "-g", label = "Recovered")
plt.ylabel("Number of people")
plt.xlabel("Days")
plt.legend()
plt.show()

print("Maximum number of infected: ", max(n_I))